In [ ]:
!pip install u8darts[torch]

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
import random

from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer

import joblib
import pickle
import os

from darts import TimeSeries
from darts.metrics import mape, smape
from darts.models import RNNModel, NBEATSModel
from darts.utils.data.sequential_dataset import SequentialDataset
from darts.utils.data.horizon_based_dataset import HorizonBasedDataset

from torch.nn import L1Loss
import torch
import torch.optim as optim

### Fixing random seeds

In [ ]:
torch.manual_seed(1337)
random.seed(1337)
np.random.seed(1337)

# Loading data

In [ ]:
df = pd.read_csv('../input/web-traffic-time-series-forecasting/train_2.csv.zip', compression='zip', index_col='Page').fillna(0)

In [ ]:
df.columns = pd.DatetimeIndex(df.columns)

# Config

In [ ]:
LOOKBACK = 3
CHUNK_OUTPUT_FINAL_LENGTH = (datetime.date(2017, 11, 13) - datetime.date(2017, 9, 10)).days
CHUNK_OUTPUT_LENGTH = 32
CHUNK_INPUT_LENGTH = CHUNK_OUTPUT_LENGTH*LOOKBACK

In [ ]:
CACHEDIR = './'
memory = joblib.Memory(CACHEDIR, verbose=0)

In [ ]:
# Cleaning cache if changing the params
!rm ./training_sequence.pickle ./test_sequence.pickle ./validation_sequence.pickle

# Training 

In [ ]:
def walk_forward_split(df: pd.DataFrame,
                       chunk_input_length: int,
                       chunk_output_length: int,
                       validset_ratio: float = 0.2,
                       testset_ratio: float = 0.1):
    
    dataset_length = df.shape[0]
    timespan = df.shape[1]

    testset_start_index = timespan - int(timespan * testset_ratio)
    validset_start_index = testset_start_index - int(timespan * validset_ratio)

    training_set = df.iloc[:, :validset_start_index]
    validation_set = df.iloc[:, validset_start_index-chunk_input_length:testset_start_index]
    test_set = df.iloc[:, testset_start_index-chunk_input_length:]

    return training_set, validation_set, test_set


def array_to_seq(arr, date_index):
    ts_sequence = []
    for i in tqdm(range(len(arr))):
        ts_sequence.append(TimeSeries.from_times_and_values(date_index, arr[i, :]))

    return ts_sequence

In [ ]:
training_set, validation_set, test_set = walk_forward_split(df=df,
                                                            chunk_input_length=CHUNK_INPUT_LENGTH,
                                                            chunk_output_length=CHUNK_OUTPUT_LENGTH,
                                                            validset_ratio=0.15,
                                                            testset_ratio=0.0)                                                            

In [ ]:
class LocalMinMaxScaler(TransformerMixin):

    def __init__(self, minimum=None):
        self.minimum = minimum

    def fit(self, X, y=None):
        if isinstance(X, pd.DataFrame):
              X = X.values

        self.min_ = X.min(axis=1).reshape(-1, 1) if self.minimum is None else self.minimum
        self.max_ = X.max(axis=1).reshape(-1, 1)

        return self

    def transform(self, X, y=None):
        return np.divide(X - self.min_, self.max_ - self.min_, out=np.zeros_like(X), where=(self.max_ - self.min_) != 0.0)

    def inverse_transform(self, X):
        return X * (self.max_ - self.min_) + self.min_

In [ ]:
class VariableSizeImputer(TransformerMixin):
    def __init__(self, fill_value=0.0):
        self.fill_value = fill_value

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill_value)

    def inverse_transform(self, X):
        return X

In [ ]:
imputer = VariableSizeImputer()
scaler = LocalMinMaxScaler(minimum=0.1)
#log_mapper = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)

In [ ]:
pipe = make_pipeline(imputer,
                     #log_mapper,
                     scaler)
pipe.fit(training_set)

In [ ]:
CACHE_ON = False

def transform_and_cache(dataset, columns, fpath):
    if CACHE_ON:
        if os.path.exists(fpath):
            return joblib.load(fpath)
        else:
            ret = array_to_seq(pipe.transform(dataset), columns)
            joblib.dump(ret, fpath)
            return ret
    else:
        return array_to_seq(pipe.transform(dataset), columns)
        

In [ ]:
training_sequence = transform_and_cache(
    training_set, 
    training_set.columns, 
    os.path.join(CACHEDIR, 'training_sequence.pickle'))

In [ ]:
validation_sequence = transform_and_cache(
    validation_set, 
    validation_set.columns,
    os.path.join(CACHEDIR, 'validation_sequence.pickle'))

In [ ]:
test_sequence = transform_and_cache(test_set, 
                                    test_set.columns,
                                    os.path.join(CACHEDIR,'test_sequence.pickle'))

In [ ]:
from darts.utils.data.horizon_based_dataset import HorizonBasedDataset

training_dataset = HorizonBasedDataset(target_series = training_sequence,
                                       output_chunk_length=CHUNK_OUTPUT_LENGTH,
                                       lh=(1,2),
                                       lookback=LOOKBACK)

validation_dataset = HorizonBasedDataset(target_series = validation_sequence,
                                         output_chunk_length=CHUNK_OUTPUT_LENGTH,
                                         lh=(1,2),
                                         lookback=LOOKBACK)

In [ ]:
def divide_no_nan(a, b):
    result = a / b
    result[result != result] = .0
    result[result == np.inf] = .0
    return result

def smape_loss(forecast, target):
    return 200 * torch.mean(divide_no_nan(torch.abs(forecast - target),
                                          torch.abs(forecast.data) + torch.abs(target.data)))

In [ ]:
N_EPOCHS = 25


NUM_STACKS=1
NUM_BLOCKS=4
NUM_LAYERS=4
LAYER_WIDTH=64
MODEL_NAME = 'NBEATS'
BATCH_SIZE = 1024

model = NBEATSModel(input_chunk_length=CHUNK_OUTPUT_LENGTH*LOOKBACK,
                    output_chunk_length=CHUNK_OUTPUT_LENGTH,
                    nr_epochs_val_period=1,
                    num_stacks=NUM_STACKS,
                    num_blocks=NUM_BLOCKS,
                    num_layers=NUM_LAYERS,
                    layer_widths=LAYER_WIDTH,
                    generic_architecture=True,
                    model_name=MODEL_NAME,
                    batch_size=BATCH_SIZE,
                    #log_tensorboard=True,
                    n_epochs=N_EPOCHS,
                    loss_fn=smape_loss)

# model fitting
print("STARTING TRAINING..")
model.fit_from_dataset(train_dataset=training_dataset, val_dataset=validation_dataset,verbose=True)

# Predictions and Post-Processing

## Reading the mapping for the competition submission format

In [ ]:
KEYS = '../input/web-traffic-time-series-forecasting/key_2.csv.zip'
def read_keys():
    keys = pd.read_csv(KEYS, compression='zip')
    id_dict = {}

    for page, page_id in zip(keys['Page'], keys['Id']):
        id_dict.update({page:page_id})

    return id_dict

In [ ]:
id_dict = read_keys()

## Predicting

In [ ]:
#model = torch.load('/content/drive/MyDrive/NBEATS_2_stacks/checkpoint_24.pth.tar')
pred = model.predict(n=CHUNK_OUTPUT_FINAL_LENGTH, series=test_sequence)

## Post-processing

In [ ]:
def map_page_and_time_to_id(page, date):
    return id_dict[page+'_'+str(date)]

In [ ]:
X = np.array([p.values().squeeze() for p in pred])

In [ ]:
post_proc_pred = pipe.inverse_transform(X)

In [ ]:
submission = []
time_index = pred[0].time_index()

for page, ts in tqdm(zip(training_set.index, post_proc_pred)):
    for timestamp, value in zip(time_index[2:], ts[2:]):
        # value[0] since values() returns an array of arrays
        submission.append([map_page_and_time_to_id(page, timestamp.date()), value])

submission_df = pd.DataFrame.from_records(submission, columns=['Id', 'Visits'])

assert len(submission_df) == 8993906

# saving the output file
submission_df.to_csv('submission.csv', index=False)
print('FILE SAVED')

In [ ]:
kaggle competitions submit -c web-traffic-time-series-forecasting -f submission.csv -m "NBEATS 1 stack, 4 blocks, 4 layers, 64 layer width, smape loss"